In [1]:
import socket
import struct
import time
import scipy.io
import numpy as np
from functools import partial
from concurrent.futures import ProcessPoolExecutor
from flwr_datasets import FederatedDataset
from flwr_datasets.partitioner import DirichletPartitioner
from torchvision.transforms import ToTensor
from flwr_datasets.visualization import plot_label_distributions
from numba import njit, jit
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from cython_decoder import cython_sc_decoding

/home/lwh/.conda/envs/wireless/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
host = '127.0.0.1'
port = 5000
server_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
server_socket.bind((host, port))
num_nodes = 5

In [ ]:
num_nodes = 20
server_socket.listen((num_nodes+1)*2)
node_s = []
node_r = []

try:
    while True:
        client_socket, addr = server_socket.accept()
        server_socket.settimeout(10)
        data = client_socket.recv(1024).decode()
        if data == "Server-R":
            server_s = client_socket
        elif data == "Server-S":
            server_r = client_socket
        elif data == "Node-R":
            node_s.append(client_socket)
        elif data == "Node-S":
            node_r.append(client_socket)
        client_socket.sendall(struct.pack('I',len(b"start"))+b"start")
except socket.timeout:
    print('Timeout')
    server_socket.settimeout(None)

for tmp_socket in node_r:
    tmp_socket.recv(1024)
server_r.recv(65536)

In [ ]:
for tmp_socket in node_r:
    tmp_socket.close()
for tmp_socket in node_s:
    tmp_socket.close()
server_s.close()
server_r.close()

In [3]:
fds = FederatedDataset(
    dataset="cifar10",
    partitioners={
        "train": DirichletPartitioner(
            num_partitions=50,
            partition_by="label",
            alpha=0.1,
            seed=42,
            min_partition_size=0,
        ),
    },
)

In [4]:
transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.Resize(32),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

transform_test = transforms.Compose([
    transforms.Resize(32),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

def train_transforms(batch):
  transforms = transform_train
  batch["img"] = [transforms(img) for img in batch["img"]]
  return batch

def test_transforms(batch):
    transforms = transform_test
    batch["img"] = [transforms(img) for img in batch["img"]]
    return batch

train_loader=[]
test_loader=[]
for i in range(50):
    partition_train_test = fds.load_partition(i, "train").train_test_split(0.1)
    partition_train = partition_train_test["train"].with_transform(train_transforms)
    partition_test = partition_train_test["test"].with_transform(test_transforms)
    # centralized_dataset = fds.load_split("test").with_transform(test_transforms)
    train_loader.append(DataLoader(partition_train, batch_size=256, shuffle=True, num_workers=16))
    test_loader.append(DataLoader(partition_test, batch_size=128, shuffle=False, num_workers=16))

In [5]:
from models.vit_small import ViT
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

net = []
optimizer = []
scheduler = []
criterion = []
scaler = []
for i in range(num_nodes):
    net.append(ViT(
        image_size = 32,
        patch_size = 4,
        num_classes = 10,
        dim = 32,
        depth = 6,
        heads = 8,
        mlp_dim = 32,
        dropout=0.1,
        emb_dropout=0.1
    ).to(device))


    optimizer.append(optim.Adam(net[i].parameters(), lr=0.001))
    scheduler.append(torch.optim.lr_scheduler.CosineAnnealingLR(optimizer[i], 5))
    criterion.append(nn.CrossEntropyLoss())
    scaler.append(torch.cuda.amp.GradScaler(enabled=True))

server_net = ViT(
    image_size = 32,
    patch_size = 4,
    num_classes = 10,
    dim = 32,
    depth = 6,
    heads = 8,
    mlp_dim = 32,
    dropout=0.1,
    emb_dropout=0.1
).to(device)

In [6]:
def train_model(model: nn.Module, 
                train_loader: DataLoader, 
                criterion: nn.Module, 
                device: torch.device, 
                scaler: torch.cuda.amp.GradScaler, 
                optimizer: torch.optim.Optimizer,
                epoch: int,
                nodes: int):
    model.train()
    total_loss = 0.0
    total_correct = 0
    total_samples = 0
    for batch in train_loader:
        inputs = batch["img"].to(device)
        labels = batch["label"].to(device)
        with torch.cuda.amp.autocast(enabled=True):
            outputs = model(inputs)
            loss = criterion(outputs, labels)
        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()
        optimizer.zero_grad()

        total_loss += loss.item()
        total_samples += labels.size(0)
        _, preds = torch.max(outputs, 1)
        total_correct += (preds == labels).sum().item()
    print(f"Nodes: {nodes}, Epoch: {epoch},Train Loss: {total_loss / total_samples:.4f}, Train Accuracy: {total_correct / total_samples:.4f}")

def evaluate_model(model: nn.Module, 
                   test_loader: DataLoader, 
                   criterion: nn.Module, 
                   device: torch.device):
    model.eval()
    total_loss = 0.0
    total_correct = 0
    total_samples = 0
    with torch.no_grad():
        for batch in test_loader:
            inputs = batch["img"].to(device)
            labels = batch["label"].to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)

            total_loss += loss.item()
            total_samples += labels.size(0)
            _, preds = torch.max(outputs, 1)
            total_correct += (preds == labels).sum().item()
    print(f"Validation Loss: {total_loss / total_samples:.4f}, Validation Accuracy: {total_correct / total_samples:.4f}\n\t")

In [7]:
for cli in range(1):
    start_time = time.time()
    for i in range(10):
        train_model(net[cli], train_loader[cli], criterion[cli], device, scaler[cli], optimizer[cli], i, cli)
        # evaluate_model(net[cli], test_loader[cli], criterion[cli], device)
        scheduler[cli].step()
    print(f"Time taken: {time.time()-start_time}")
    scheduler[cli] = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer[cli], 5)
    

Nodes: 0, Epoch: 0,Train Loss: 0.0077, Train Accuracy: 0.3586
Nodes: 0, Epoch: 1,Train Loss: 0.0065, Train Accuracy: 0.4246
Nodes: 0, Epoch: 2,Train Loss: 0.0063, Train Accuracy: 0.4132
Nodes: 0, Epoch: 3,Train Loss: 0.0061, Train Accuracy: 0.4126
Nodes: 0, Epoch: 4,Train Loss: 0.0060, Train Accuracy: 0.4294
Nodes: 0, Epoch: 5,Train Loss: 0.0061, Train Accuracy: 0.4168
Nodes: 0, Epoch: 6,Train Loss: 0.0060, Train Accuracy: 0.4252
Nodes: 0, Epoch: 7,Train Loss: 0.0060, Train Accuracy: 0.4222
Nodes: 0, Epoch: 8,Train Loss: 0.0060, Train Accuracy: 0.4258
Nodes: 0, Epoch: 9,Train Loss: 0.0059, Train Accuracy: 0.4144
Time taken: 3.633261203765869


In [11]:
def numpy_to_bit(array: np.ndarray) -> np.ndarray:
    tmp_byte=np.frombuffer(array.tobytes(),dtype=np.uint8)
    bit_stream = ''.join(format(byte, '08b') for byte in tmp_byte)
    bit_array = np.array([int(bit) for bit in bit_stream], dtype=np.int8)
    return bit_array

def bit_to_numpy(bit_array: np.ndarray) -> np.ndarray:
    bit_stream = ''.join(str(int(bit)) for bit in bit_array)
    byte_array_back = np.array([int(bit_stream[i:i+8], 2) for i in range(0, len(bit_stream), 8)], dtype=np.uint8)
    int_array_back = np.frombuffer(byte_array_back.tobytes(), dtype=np.float32)
    return int_array_back

'''
Encoding and decoding function
'''
@jit(nopython=True)
def encode(u: np.ndarray) -> np.ndarray:
    N = u.shape[0]  # Get the length of u
    n = int(np.log2(N))  # Calculate the log base 2 of N

    if n == 1:
        x = np.array([(u[0] + u[1]) % 2, u[1]],dtype=np.int8)
        return x
    else:
        x1 = encode(np.mod(u[:N//2] + u[N//2:], 2))
        x2 = encode(u[N//2:])
        x = np.concatenate((x1, x2))
        return x

@jit(nopython=True)
def rvsl(y: np.ndarray) -> np.ndarray:
    N = y.shape[0]
    if N == 2:
        return y
    else:
        return np.concatenate((rvsl(y[0:N:2]), rvsl(y[1:N:2])))

def data_process(array):
    bit_array = numpy_to_bit(array)
    array_len = len(bit_array)
    current_array = []
    for i in range(0, array_len, 512):
        sub_array = bit_array[i:i+512]
        if len(sub_array) < 512:
            padding = np.ones((512 - len(sub_array)), dtype=bit_array.dtype)
            sub_array = np.concatenate((sub_array, padding))
        current_array.append(sub_array)
    current_idx = i // 512 + 1
    return current_array, current_idx, array_len

def numpy_array_to_udp_packet(bit_array):
    # Convert the numpy array of bits to a string of bits
    bit_string = ''.join(str(int(bit)) for bit in bit_array)
    # Convert the bit string to bytes
    byte_array = bytearray(int(bit_string[i:i+8], 2) for i in range(0, len(bit_string), 8))
    return byte_array

def udp_packet_to_numpy_array(packet):
    # Convert the byte array back to a bit string
    bit_string = ''.join(format(byte, '08b') for byte in packet)
    # Convert the bit string to a numpy array of floats
    bit_array = np.array([int(bit) for bit in bit_string], dtype=np.int8)
    return bit_array

def data_generate(bit_array:np.ndarray, data_idx:np.ndarray) -> np.ndarray:
    u=np.zeros(1024,dtype=np.int8)
    u[data_idx] = bit_array
    x = encode(u)
    x = rvsl(x)
    return x

def codeword_generate(array_dict, data_idx):
    split_bit = []
    codeword_idx = []
    bit_array_len = []
    codeword_idx.append(0)
    array_process = partial(data_process)
    with ProcessPoolExecutor() as executor:
        for current_array, current_idx, array_len in executor.map(array_process, [array_dict[name] for name in array_dict]):
            split_bit.extend(current_array)
            codeword_idx.append(codeword_idx[-1] + current_idx)
            bit_array_len.append(array_len)
    executor.shutdown(wait=True)
    del executor, array_process
    time1 = time.time()
    encode_partial = partial(data_generate, data_idx=data_idx)
    with ProcessPoolExecutor() as executor:
        codeword = list(executor.map(encode_partial, split_bit))
    executor.shutdown(wait=True)
    time2 = time.time()
    print(f'Encode time: {time2-time1}')

    del executor, encode_partial
    return np.array(codeword,dtype=np.int8), codeword_idx, bit_array_len

def packet_diffusion(codeword, block_len, packet_idx):
    codeword_len = codeword[0].shape[0]
    udp_packet = []
    for idx, i in enumerate(range(0, codeword_len, block_len)):
        # tmp_packet = np.concatenate([tmp_codeword[i:i+block_len] for tmp_codeword in codeword])
        tmp_packet = codeword[:, packet_idx[i:i+block_len]].flatten()
        tmp_udp_packet = struct.pack("I",idx) + numpy_array_to_udp_packet(tmp_packet)
        udp_packet.append(tmp_udp_packet)
    return udp_packet

def encoder_udp(array_dict, data_idx, block_len, packet_idx):
    codeword, codeword_idx, bit_array_len = codeword_generate(array_dict, data_idx)
    udp_packet = packet_diffusion(codeword, block_len, packet_idx)
    return udp_packet, codeword_idx, bit_array_len



def packet_aggregation(udp_packet, packet_idx, block_len, data_idx, freeze_idx, codeword_idx, bit_array_len):
    sort_idx = [struct.unpack("I", tmp_packet[:4])[0] for tmp_packet in udp_packet]
    packet_data_del = np.array([udp_packet_to_numpy_array(tmp_packet[4:]) for _, tmp_packet in sorted(zip(sort_idx, udp_packet))])
    packet_data = np.ones((int(1024/block_len), len(packet_data_del[0])))*0.5
    for i, tmp_idx in enumerate(sorted(sort_idx)):
        packet_data[tmp_idx] = packet_data_del[i]
    
    restore_codeword = []
    inverse_packet_idx = np.argsort(packet_idx)
    for i in range(0, packet_data.shape[1],block_len):
        tmp_codeword = packet_data[:,i:i+block_len].flatten()
        restore_codeword.append(tmp_codeword[inverse_packet_idx])

    decode_partial = partial(decoding, freeze_idx=freeze_idx, data_idx=data_idx)
    with ProcessPoolExecutor() as executor:
        decoding_data = np.array(list(executor.map(decode_partial, restore_codeword)),dtype=np.int8)
    del executor, decode_partial

    restore_array = []
    for i, array_len in enumerate(bit_array_len):
        tmp_array = np.concatenate(decoding_data[codeword_idx[i]:codeword_idx[i+1]])[:array_len]
        restore_array.append(bit_to_numpy(tmp_array))
    return restore_array


def decoding(bit_array, freeze_idx, data_idx):
    # Prepare the necessary arrays and values
    bit_array = 1-2*bit_array
    lr0 = np.exp(-(bit_array - 1)**2)
    lr1 = np.exp(-(bit_array + 1)**2)
    lr0_post = lr0 / (lr0 + lr1)
    lr1_post = lr1 / (lr0 + lr1)
    delete_num = 1024 - len(bit_array)
    hd_dec = np.zeros(1024, dtype=np.float64)
    frozen_val = np.zeros(len(freeze_idx), dtype=np.float64)
    pro_prun = np.zeros((1, 2 * 1024 + 1), dtype=np.float64)

    # Call the optimized Cython function
    i_scen_sum, hd_dec_result = cython_sc_decoding(
        lr0_post, lr1_post, freeze_idx.astype(np.float64),
        hd_dec, 1024, 10, 512, frozen_val, delete_num, 0, pro_prun
    )

    # Extract the output for data_idx from hd_dec_result
    data_out = hd_dec_result[data_idx]
    return data_out


In [9]:
# Export all weights to numpy arrays
weights_dict = {name: param.cpu().detach().numpy() for name, param in net[0].state_dict().items()}
N = 1024
n = 10
rate = 0.5
K = round(N*rate)
c_1024 = np.load('c_1024.npy')
coding_list = scipy.io.loadmat("1024-3db-d=2-mean.mat")["count_number"]
coding_index = np.argsort(coding_list[:,1])
info_idx = coding_index[:K]
freeze_idx = coding_index[K:]

# sort the final index
info_ni = np.sort(info_idx)
freeze_ni = np.sort(freeze_idx)

In [477]:
udp_packet, codeword_idx, bit_array_len = encoder_udp(weights_dict, info_ni, 8, c_1024)
restored_array = packet_aggregation(udp_packet, c_1024, 8, info_ni, freeze_ni, codeword_idx, bit_array_len)
restored_dict = {}
# restored_array = [torch.tensor(arr).to(device) for arr in restored_array]
for i, name in enumerate(weights_dict):
    restored_dict[name] = torch.tensor(restored_array[i].reshape(weights_dict[name].shape)).to(device)

In [12]:
def get_gn(length:int, device):
    n = int(np.log2(length))
    f=np.array([[1,0],[1,1]], dtype=bool)
    g_n = f
    for _ in range(n-1):
        g_n = np.kron(g_n, f)
    g_order = rvsl(np.arange(length))
    g_n = g_n[:, g_order]
    g_n = torch.tensor(g_n, dtype=torch.float32).to(device)
    return g_n

def rvsl(y: np.ndarray) -> np.ndarray:
    N = y.shape[0]
    if N == 2:
        return y
    else:
        return np.concatenate((rvsl(y[0:N:2]), rvsl(y[1:N:2])))

def data_process(array, data_idx):
    bit_array = np.frombuffer(array.tobytes(), dtype=np.uint8)
    info_len = int(len(data_idx) / 8)
    array_len = bit_array.shape[0] * 8
    if bit_array.shape[0] % info_len != 0:
        padding = 255 * np.ones((info_len - bit_array.shape[0] % info_len), dtype=bit_array.dtype)
        bit_array = np.concatenate((bit_array, padding))
    bit_array = np.unpackbits(bit_array)
    bit_array = bit_array.reshape(-1, len(data_idx))
    current_array = np.zeros((bit_array.shape[0], 1024), dtype=np.uint8)
    current_array[:, data_idx] = bit_array
    current_idx = bit_array.shape[0]
    return current_array, current_idx, array_len

def udp_packet_to_numpy_array(packet, codeword_num):
    bit_array = np.frombuffer(packet, dtype=np.uint8)
    bit_array = np.unpackbits(bit_array)
    bit_array = bit_array[:codeword_num]
    return bit_array

def codeword_generate(array_dict, data_idx, gn):
    split_bit = []
    codeword_idx = []
    bit_array_len = []
    codeword_idx.append(0)
    for name in array_dict:
        current_array, current_idx, array_len = data_process(array_dict[name], data_idx)
        split_bit.extend(current_array)
        codeword_idx.append(codeword_idx[-1] + current_idx)
        bit_array_len.append(array_len)
    split_bit = torch.tensor(np.array(split_bit), dtype=torch.float32).to("cuda:0")
    tmp_codeword = torch.matmul(split_bit, gn) % 2
    codeword = np.array(tmp_codeword.cpu().detach().numpy(), dtype=np.int8)
    return codeword, codeword_idx, bit_array_len

def packet_diffusion(codeword):
    if codeword.shape[0] % 8 != 0:
        padding = np.ones((8 - codeword.shape[0] % 8, 1024), dtype=codeword.dtype)
        udp_numpy = np.concatenate((codeword, padding)).T
    udp_numpy = np.packbits(udp_numpy.flatten()).reshape(1024,-1)
    udp_packet = [struct.pack("I", idx) + udp_numpy[idx].tobytes() for idx in range(udp_numpy.shape[0])]
    return udp_packet

def encoder_udp(array_dict, data_idx, gn):
    codeword, codeword_idx, bit_array_len = codeword_generate(array_dict, data_idx, gn)
    udp_packet = packet_diffusion(codeword)
    return udp_packet, codeword_idx, bit_array_len, codeword.shape[0]

def packet_aggregation(udp_packet, codeword_num, data_idx, freeze_idx, codeword_idx, bit_array_len):
    udp_idx = []
    packet_del = []
    for tmp_packet in udp_packet:
        udp_idx.append(struct.unpack("I", tmp_packet[:4])[0])
        packet_del.append(udp_packet_to_numpy_array(tmp_packet[4:], codeword_num))
    packet_del = np.array(packet_del)
    packet_data = np.ones((1024, codeword_num)) * 0.5
    packet_data[udp_idx] = packet_del

    restore_codeword = packet_data.T
    decode_partial = partial(decoding, freeze_idx=freeze_idx, data_idx=data_idx)
    with ProcessPoolExecutor() as executor:
        decoding_data = np.array(list(executor.map(decode_partial, restore_codeword)),dtype=np.int8)
    del executor, decode_partial
    restore_array = []
    for i, array_len in enumerate(bit_array_len):
        tmp_array = np.concatenate(decoding_data[codeword_idx[i]:codeword_idx[i+1]])[:array_len]
        bit_array = np.packbits(tmp_array)
        restore_array.append(np.frombuffer(bit_array.tobytes(), dtype=np.float32))
    return restore_array


def decoding(bit_array, freeze_idx, data_idx):
    # Prepare the necessary arrays and values
    bit_array = 1-2*bit_array
    lr0 = np.exp(-(bit_array - 1)**2)
    lr1 = np.exp(-(bit_array + 1)**2)
    lr0_post = lr0 / (lr0 + lr1)
    lr1_post = lr1 / (lr0 + lr1)
    delete_num = 1024 - len(bit_array)
    hd_dec = np.zeros(1024, dtype=np.float64)
    frozen_val = np.zeros(len(freeze_idx), dtype=np.float64)
    pro_prun = np.zeros((1, 2 * 1024 + 1), dtype=np.float64)

    # Call the optimized Cython function
    i_scen_sum, hd_dec_result = cython_sc_decoding(
        lr0_post, lr1_post, freeze_idx.astype(np.float64),
        hd_dec, 1024, 10, len(freeze_idx), frozen_val, delete_num, 0, pro_prun
    )

    # Extract the output for data_idx from hd_dec_result
    data_out = hd_dec_result[data_idx]
    return data_out


In [35]:
# Export all weights to numpy arrays
weights_dict = {name: param.cpu().detach().numpy() for name, param in net[0].state_dict().items()}
N = 1024
n = 10
rate = 1/8
K = round(N*rate)
c_1024 = np.load('c_1024.npy')
coding_list = scipy.io.loadmat("1024-3db-d=2-mean.mat")["count_number"]
coding_index = np.argsort(coding_list[:,1])
info_idx = coding_index[:K]
freeze_idx = coding_index[K:]

# sort the final index
info_ni = np.sort(info_idx)
freeze_ni = np.sort(freeze_idx)
gn = get_gn(1024, "cuda:0")

In [36]:
udp_packet, codeword_idx, bit_array_len, codeword_num = encoder_udp(weights_dict, info_ni, gn)

In [20]:
restore_array = packet_aggregation(udp_packet, codeword_num, info_ni, freeze_ni, codeword_idx, bit_array_len)

In [67]:
lost_rate = 0.65
received_idx = np.random.choice(1024, int(1024*(1-lost_rate)), replace=False)
received_packet = [udp_packet[idx] for idx in received_idx]
restore_array = packet_aggregation(received_packet, codeword_num, info_ni, freeze_ni, codeword_idx, bit_array_len)

In [68]:
restore_array

[array([ 0.5310353 ,  0.5830858 ,  0.47036096, ...,  0.23929603,
         0.49191788, -0.2649631 ], dtype=float32),
 array([ 0.89204943, -0.402101  ,  0.63489187,  1.847776  , -1.0532882 ,
         0.79509294,  0.19724345, -1.4859813 ,  0.9266959 ,  0.25420916,
         0.87743175,  0.8530794 ,  0.8633613 , -0.20926446,  1.3015786 ,
         0.60156125, -0.5701518 , -1.5352505 ,  1.6077627 ,  1.6693171 ,
         0.00530217, -0.52378994, -0.0441067 ,  1.6146165 , -0.06558976,
         0.8671432 ,  1.4782531 ,  0.65860707,  0.41266146,  2.0960276 ,
         0.5138869 , -0.24219371], dtype=float32),
 array([0.9936637 , 1.0015844 , 0.9998757 , 0.9947306 , 1.0029247 ,
        0.9921213 , 0.9934131 , 0.9888145 , 1.0124007 , 0.992486  ,
        1.013585  , 1.0070069 , 0.9935403 , 0.9987652 , 1.0075698 ,
        0.9883472 , 1.0029188 , 0.9957118 , 0.9960165 , 1.0122418 ,
        1.0107763 , 0.9945393 , 1.0026048 , 0.9908437 , 0.9986736 ,
        1.0136243 , 1.0081089 , 0.99769366, 1.0024225 ,

In [41]:
weights_dict

{'pos_embedding': array([[[ 0.5310353 ,  0.5830858 ,  0.47036096, ..., -2.7078679 ,
          -1.7308646 , -0.28650394],
         [ 1.0992341 , -0.49056298, -0.08798212, ..., -0.76179874,
          -0.55762005, -1.4989058 ],
         [-0.3569383 , -1.4169679 , -0.5548522 , ..., -0.68567955,
          -0.34508476, -2.8303483 ],
         ...,
         [ 0.77762383,  0.01972509, -0.6726472 , ...,  0.4802725 ,
          -0.31741422,  0.49853235],
         [ 0.4218498 , -0.22112392,  0.3494972 , ..., -0.57955897,
          -0.08198946, -1.047114  ],
         [ 1.2308822 ,  1.6802766 , -1.1143656 , ...,  0.23929603,
           0.49191788, -0.2649631 ]]], dtype=float32),
 'cls_token': array([[[ 0.89204943, -0.402101  ,  0.63489187,  1.847776  ,
          -1.0532882 ,  0.79509294,  0.19724345, -1.4859813 ,
           0.9266959 ,  0.25420916,  0.87743175,  0.8530794 ,
           0.8633613 , -0.20926446,  1.3015786 ,  0.60156125,
          -0.5701518 , -1.5352505 ,  1.6077627 ,  1.6693171 ,
    